## Instrucciones

Edita, elabora y ejecuta los comandos y responde a las preguntas en este mismo documento. Pero **¡atención!** debes descargarlo (en formato `.ipynb`) para poder entregarlo a través del aula virtual.

## Problema 1
En la carpeta `data` hay cuatro ficheros en formato FASTQ, con secuencias de cuatro muestras diferentes de cerbero común (*Canis cerberus*). El objetivo es: primero realizar un análisis de la calidad de las secuencias y después mapearlas sobre el genoma de referencia, `data/cerber.fa`. Pero los bloques de código siguientes están desordenados y contienen errores. Corrige los errores y ejecútalos en el orden correcto para poder responder las preguntas siguietes.

In [ ]:
# BLOQUE METBREWER

library('MetBrewer')
system2(command = 'cut', args = c('-f', '4,5', 'BRZ026.sam'), stdout = 'MapQualBRZ026.txt')
system2(command = 'cut', args = c('-f', '4,5', 'BRZ031.sam'), stdout = 'MapQualBRZ031.txt')
system2(command = 'cut', args = c('-f', '4,5', 'LAN048.sam'), stdout = 'MapQualLAN048.txt')
system2(command = 'cut', args = c('-f', '4,5', 'LAN098.sam'), stdout = 'MapQualLAN098.txt')
mapq.BRZ026 <- read.table('MapQualBRZ026.txt', col.names = c('pos','mapq'))
mapq.BRZ031 <- read.table('MapQualBRZ031.txt', col.names = c('pos','mapq'))
mapq.LAN048 <- read.table('MapQualLAN048.txt', col.names = c('pos','mapq')
mapq.LAN098 <- read.table('MapQualLAN098.txt' col.names = c('pos','mapq'))
colors = met.brewer(name = 'Egypt', n = 4)
plot(density(mapq.BRZ031$mapq),  lwd = 2, col = colors[1], xlab = 'Calidad del mapeo', ylab = 'Densidad', main = '')
lines(density(mapq.BRZ026$mapq), lwd = 2, col = colors[2])
lines(density(mapq.LAN048$mapq), lwd = 2, col = colors[3])
lines(density(mapq.LAN098$mapq), lwd = 2, col = colors[4])
legend(10, y=0.08, legend = c('BRZ031','BRZ026','LAN048','LAN098'), col = colors, lwd = 2)

In [ ]:
# BLOQUE BOWTIE
BRZ026.out <- bowtie2(bt2Index = 'index/cerber',
                     samOutput = 'BRZ026.sam',
                          seq1 = 'data/BRZ026.fq'
                     overwrite = TRUE,
                     '--no-unal', '--no-head')
BRZ026.out

BRZ031.out <- bowtie2(bt2Index = 'index/cerber',
                     samOutput = 'BRZ031.sam',
                          seq1 = 'data/BRZ031.fa',
                     overwrite = TRUE,
                     '--no-unal', '--no-head')
BRZ031.out

LAN048.out <  bowtie2(bt2Index = 'index/cerber',
                     samOutput = 'LAN048.sam',
                          seq1 = 'data/LAN048.fq',
                     overwrite = TRUE,
                     '--no-unal', '--no-head')
LAN048.out

LAN098.out <- bowtie2(bt2Index = 'index/cerber',
                     samOutput = 'LAN098.sam',
                          seq1 = 'data/LAN098.fq',
                     overwrite = TRUE,
                     '--no-unal', '--no-head')
LAN098.out

In [ ]:
# BLOQUE GGPLOT2

library(ggplot2)
suppressMessages(library('ShortRead'))
ResumenCalidad <- qa('data', type = 'fastq', pattern = '*.fa')
ResumenCalidad
head(ResumenCalidad[['baseCalls']])
report(ResumenCalidad, dest = 'ResumenCalidad.html')
ggplot(data = ResumenCalidad[['readQualityScore']],
       mapping = aes(x = quality, y = density)) +
   geom_line() +
   facet_wrap(~lane)

In [ ]:
# BLOQUE RBOWTIE2

library('Rbowtie2')
dir.create('index')
bowtie2_build(references = 'data/cerber.fq',
              bt2Index   = 'index/cerbero',
              '--quiet', overwrite = TRUE)

Contesta las preguntas siguientes:

1. Indica en qué orden crees que deben ejecutarse los pasos.
2. ¿Cuántas lecturas cortas hay en cada archivo?
3. ¿Crees que las lecturas cortas son emparejadas (*paired ends*) o no?
4. ¿Qué longitudes tienen las lecturas?
2. Se sospecha que una de las tres muestras no pertenece a la misma especie que las otras. ¿Cuál dirías que és?

## Problema 2



In [ ]:
library(curl)
portal   <- 'https://www.ebi.ac.uk/ena/portal/api/'
endpoint <- 'search?'
result   <- 'result=read_run'
query    <- paste0('&query=',
                   URLencode('tax_tree(2) AND first_public>=2021-05-01',
                             reserved=TRUE))
fields   <- '&fields=scientific_name,read_count,base_count,library_source,center_name,country'
limit    <- '&limit=0'

URL <- paste0(portal, endpoint, result, query, fields, limit, sep='', collapse='')
curl_download(URL, destfile = 'descarga1.tsv')
resultados <- read.table('descarga1.tsv', header = TRUE, quote = "\"",
                         sep = '\t', na.string = '', stringsAsFactors = FALSE)
head(resultados)
dim(resultados)
sum(resultados$read_count, na.rm = TRUE)
sum(resultados$base_count, na.rm = TRUE)
resultados$genero <- sapply(resultados$scientific_name, function(x) strsplit(x, ' ')[[1]][1])
head(resultados)
sort(table(resultados$genero), decreasing = TRUE)
resultados$country <- sapply(resultados$country, function(x) strsplit(x, ':')[[1]][1])
sort(table(resultados$country), decreasing = TRUE)
sort(table(resultados[resultados$country == 'Russia', 'scientific_name']), decreasing = TRUE)
sort(table(resultados[resultados$country == 'Ireland', 'center_name']), decreasing = TRUE)

#### Preguntas sobre el análisis de ejemplo anterior

1. ¿Cuál es el número de registros de experimentos de secuenciación con lecturas cortas publicados en ENA durante este mes de mayo?

2. ¿Cuál es el género de Bacterias más secuenciado mediante lecturas cortas en el mes de mayo del presente año?

3. ¿Qué especies de bacterias se han dedicado a secuenciar en Rusia durante el mes de mayo? ¿Y en España?

4. ¿Cuántos centros de investigación españoles diferentes han enviado secuencias cortas de Bacterias a la base de datos ENA durante este mes? ¿E irlandeses? 


####  Problema
Elabora ahora tu propia consulta, en un bloque de código nuevo, para investigar los estudios de secuenciación con lecturas cortas en el ratón (*Mus musculus*, taxid 10090) durante el mes de mayo del presenta año. Redacta un pequeño resumen con tus conclusiones. Deberías intentar incluir información sobre: los tipos de tejidos o líneas celulares más secuenciados, los tipos de estudio (*library_source*), el balance entre ratones macho y hembra secuenciados, así como cualquier otro aspecto que te llame la atención. Si prefieres realizar el análisis en [la web de ENA](https://www.ebi.ac.uk/ena/browser/home), o bien en una hoja de cálculo después de descargar los resultados de la búsqueda, también podrás obtener la nota máxima en este apartado. No deberías demorarte más de 20 minutos.


## Problema 3

Para poder realizar este problema, necesitas haber instalado los programas de blast del NCBI. Para hacerlo, solo tienes que ejecutar el código siguiente. Puede tardar unos segundos.

In [ ]:
system2(command = './preparar_ambiente.sh', wait = TRUE)

Dentro de la carptea `data` tienes un archivo fasta llamado `a.fa`. El objetivo es realizar algunas búsquedas de blast con esta *query*. Puedes basarte en los comandos del bloque siguiente, siempre que puedas corregir los errores antes.

1. Busca la proteína **a** en la base de datos *swissprot* (instalada en la carpeta actual) mediante **blastp**, utilizando un umbral de valor E de 0.001. Anota el número de resultados obtenidos.
3. Conociendo la definición del "valor E" y a la vista de los resultados de la búsqueda, ¿sabrías decir cuántos de los resultados obtenidos esperas que sean casuales (es decir, que igualmente podrían haberse producido al azar)?
4. Realiza ahora una búsqueda con **psiblast**, en la misma base de datos y con la misma proteína de partida. ¿Cuántas iteraciones necesitas para alcanzar la convergencia si usas un umbral de inclusión (`-inclusion_ethresh`) de 0.001?
5. Compara el número de resultados obtenidos con **psiblast** con el número de resultados obtenidos antes con **blastp** y explica a qué se debe la diferencia.

In [ ]:
system2(command = 'blastn',
        args = c('-db', 'swissprot',
                 '-query', 'data/b.fa',
                 '-out', 'psi1.txt',
                 '-outfmt', '"7 qaccver saccver pident length qstart qend sstart send
   evalue ssciname sblastname"',
                 '-evalue', '0.005',
                 '-inclusion_ethresh', '0.002',
                 '-num_iterations', '0'),
       stdout=TRUE)